In [1]:
from ga4stpg.graph import UGraph

In [2]:
from collections import deque

In [3]:
edges = [
    (1, 3),
    (1, 15),
    (15, 17),
    (3, 5),
    (5, 11),
    (11, 13),
    (5, 7),
    (7, 9)
]

graph = UGraph()

for v, u in edges:
    graph.add_edge(v, u)

In [4]:
# def find_segments(graph, common_nodes, start):
#     visited = set()
#     stack_outter = [start]
#     result = list()

#     def search(start, neighbor):
#         segment = set()
#         segment.add( (start, neighbor) )
#         visited.add(start)
#         stack_inner = [neighbor]

#         while stack_inner:
#             u = stack_inner.pop()
#             visited.add(u)
#             if u not in common_nodes:
#                 for w in graph.adjacent_to(u):
#                     if w not in visited:
#                         stack_inner.append(w)
#                         segment.add((u, w))
#             else:
#                 stack_outter.append(u)
#         # end while
#         return segment
#         # end search

#     while stack_outter:
#         s = stack_outter.pop()
#         visited.add(s)
#         for v in graph.adjacent_to(s):
#             if v not in visited:
#                 seg = search(s, v) 
#                 result.append(seg)

#     return result

In [5]:
common_nodes = {1, 17, 5, 13, 9}
start = 1

In [6]:
# find_segments(graph, common_nodes, start)

## Segundo Exemplo

In [7]:
edges = [
    (1, 3),
    (1, 15),
    (15, 17),
    (3, 5),
    (5, 11),
    (11, 13),
    (5, 7),
    (7, 9),
    (7, 23)
]

graph = UGraph()

for v, u in edges:
    graph.add_edge(v, u)

In [8]:
# common_nodes = {1, 17, 5, 13, 9, 23}
# start = 17

# seg = find_segments(graph, common_nodes, start)

# for s in seg:
#     print(s)

## Identificando partições para cruzamento de duas soluções

In [9]:
from os import path

from ga4stpg.graph.disjointsets import DisjointSets
from ga4stpg.graph.priorityqueue import PriorityQueue
from ga4stpg.graph.reader import read_problem
from ga4stpg.util import STEIN_B
from ga4stpg.graph.util import has_cycle, how_many_components

from ga4stpg.tree.evaluation import EvaluateTreeGraph
from ga4stpg.tree.generate import GenerateBasedPrimRST
from ga4stpg.tree.mutate import ReplaceByRandomEdge
from ga4stpg.tree.pxcrossover import PXTree

In [10]:
INDEX_SELECT = 4

datasets = path.join('..', '..', 'ppgi-stpg-gpx', 'datasets', 'ORLibrary')

filename = path.join(datasets, STEIN_B[INDEX_SELECT][0])

print(filename)

STPG = read_problem(filename)

..\..\ppgi-stpg-gpx\datasets\ORLibrary\steinb5.txt


In [11]:
from ga4stpg.edgeset import UEdge, EdgeSet

class Segment:

    def __init__(self):
        self.edges = EdgeSet()
        self.cost = 0
        self.portals = set()

    def __len__(self):
        return len(self.edges)

    def __str__(self):
        return f'Segment <{len(self.edges)}>'

    def __iter__(self):
        return iter(self.edges)

    @property
    def bounds(self):
        return frozenset(self.portals)
    
    def add(self, v, u):
        self.edges.add(UEdge(v, u))
    

In [12]:
def f_weight(v, u):
    if STPG.graph.has_edge(v, u):
        return STPG.graph.weight(v, u)
    else:
        return 0

def find_segments(graph, common_nodes):
    visited = set()
    stack_outter = list(common_nodes)
    result = list()

    def search(start, neighbor):
        segment = Segment()
        segment.add(start, neighbor)
        segment.cost += f_weight(start, neighbor)
        segment.portals.add(start)
        
        stack_inner = [neighbor]

        while stack_inner:
            u = stack_inner.pop()
            visited.add(u)
            if u not in common_nodes:
                for w in graph.adjacent_to(u):
                    if w not in visited:
                        stack_inner.append(w)
                        segment.add(u, w)
                        segment.cost += f_weight(u, w)
                    if len(stack_inner) == 0:
                        segment.portals.add(w)
            else:
                stack_outter.append(u)
                segment.portals.add(u)
        # end while
        return segment
        # end search

    while stack_outter:
        s = stack_outter.pop()
        visited.add(s)
        for v in graph.adjacent_to(s):
            if v not in visited:
                visited.add(s)
                seg = search(s, v)
                result.append(seg)

    return result

In [13]:
generator = GenerateBasedPrimRST(STPG)

red  = generator()
blue = generator()

In [14]:
red.edges == blue.edges

False

In [15]:
child     = UGraph()
red_only  = UGraph()
blue_only = UGraph()

In [16]:
for v, u in red.gen_undirect_edges():
    if blue.has_edge(v, u):
        child.add_edge(v, u)
    else:
        red_only.add_edge(v, u)

In [17]:
for v, u in blue.gen_undirect_edges():
    if not red.has_edge(v, u):
        blue_only.add_edge(v, u)

In [18]:
how_many_components(child)

7

In [19]:
common_nodes = set(red_only.vertices) & set(blue_only.vertices)
terminals = STPG.terminals.copy()

common_nodes & terminals

{3, 24, 39}

In [20]:
len(common_nodes)

20

In [21]:
red_segments  = find_segments(red_only, common_nodes)
blue_segments = find_segments(blue_only, common_nodes)

len(red_segments)

14

In [22]:
len(blue_segments)

16

In [23]:
queue = PriorityQueue()

for seg in red_segments:
    # print(seg, seg.cost)
    queue.push(seg.cost, seg)

for seg in blue_segments:
    # print(seg, seg.cost)
    queue.push(seg.cost, seg)

len(queue)

30

In [24]:
dsets = DisjointSets()

for v in child.vertices:
    dsets.make_set(v)

for v in common_nodes:
    if v not in dsets:
        dsets.make_set(v)

for v, u in child.gen_undirect_edges():
    dsets.union(v, u)

In [25]:
dsets.get_disjoint_sets()

defaultdict(set,
            {26: {3, 4, 5, 15, 20, 25, 26, 29, 31, 48, 49},
             42: {6, 7, 11, 12, 16, 22, 23, 32, 33, 35, 42, 44, 46, 47, 50},
             40: {1, 40},
             8: {8, 36},
             21: {9, 21},
             37: {19, 37},
             13: {10, 13},
             2: {2},
             14: {14},
             17: {17},
             18: {18},
             24: {24},
             27: {27},
             28: {28},
             30: {30},
             34: {34},
             38: {38},
             39: {39},
             43: {43},
             45: {45}})

In [26]:
def check_portals(portals, disjoint):
    '''Verifica se os vértices portais de um segmento
    se conectam à mesma partição de vértices comuns. 

    Faz essa verificação em tempo O(n)
    '''
    f_check = set()

    for p in portals:
        k = disjoint.find(p)
        if k in f_check:
            return False
        f_check.add(k)

    return True

In [27]:
while queue:
    seg = queue.pop()
    test = set()

    if len(seg.portals) == 1:
        continue

    if check_portals(seg.portals, dsets):
        # print('adding segment', seg)
        for edge in seg:
            v, u = edge 
            child.add_edge(v, u)
        portals = list(seg.portals)
        for p1, p2 in zip(portals[0::2], portals[1::2]):
            dsets.union(p1, p2)
            print(p1, p2, dsets.find(p1), dsets.find(p2))
    else:
        pass
        # print("####### >>> rejecting: ", seg)

24 4 26 26
10 18 13 13
3 14 26 26
36 45 8 8
3 39 26 26
34 3 26 26
45 38 8 8
28 30 30 30
17 27 27 27
28 4 26 26
42 45 8 8
9 42 8 8
24 36 8 8
10 19 37 37
42 27 8 8
2 28 8 8
19 39 8 8


In [28]:
for seg in red_segments:
    parent = dict()
    for p in seg.portals:
        parent[p] = dsets.find(p)

    print(parent)

{42: 8, 45: 8}
{36: 8, 45: 8}
{45: 8, 38: 8}
{9: 8, 43: 43, 30: 8, 14: 8}
{28: 8, 30: 8}
{3: 8, 39: 8}
{34: 8, 3: 8}
{27: 8}
{24: 8, 4: 8}
{10: 8, 19: 8}
{19: 8}
{18: 8}
{17: 8}
{2: 8}


In [29]:
for seg in blue_segments:
    parent = dict()
    for p in seg.portals:
        parent[p] = dsets.find(p)

    print(parent)

{45: 8}
{10: 8, 43: 43, 34: 8}
{10: 8}
{10: 8, 18: 8}
{9: 8, 42: 8}
{42: 8, 27: 8}
{42: 8, 38: 8}
{17: 8, 27: 8}
{9: 8, 36: 8}
{24: 8, 36: 8}
{19: 8, 39: 8}
{30: 8}
{2: 8, 28: 8}
{28: 8, 4: 8}
{4: 8}
{3: 8, 14: 8}


In [30]:
has_cycle(child)

True

In [31]:
how_many_components(child)

2

In [32]:
seg

In [33]:
for seg in blue_segments:
    print(seg.portals)
    for edge in seg:
        print(edge, end='  ')
    print("")

{45}
Edge <(8, 45)>  
{10, 43, 34}
Edge <(10, 23)>  Edge <(23, 43)>  Edge <(23, 34)>  
{10}
Edge <(10, 40)>  
{10, 18}
Edge <(10, 18)>  
{9, 42}
Edge <(9, 42)>  
{42, 27}
Edge <(27, 42)>  
{42, 38}
Edge <(38, 49)>  Edge <(42, 49)>  
{17, 27}
Edge <(17, 27)>  
{9, 36}
Edge <(9, 36)>  
{24, 36}
Edge <(24, 36)>  
{19, 39}
Edge <(20, 39)>  Edge <(19, 20)>  
{30}
Edge <(11, 30)>  
{2, 28}
Edge <(2, 28)>  
{28, 4}
Edge <(4, 28)>  
{4}
Edge <(4, 41)>  
{3, 14}
Edge <(3, 14)>  


In [34]:
4 in common_nodes

True

In [35]:
41 in common_nodes

False